<a href="https://colab.research.google.com/github/survivingwithandroid/Animal_Sound_Dataset_Edge_Impulse/blob/main/Arduino_Animal_Sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the Audio dataset from Kaggle

In [ ]:
!pip install -q kaggle
import os
os.environ['KAGGLE_USERNAME']='francescoazzola'
os.environ['KAGGLE_KEY']='1e47af3c13cb61af7f0ce955f79ecc63'


# Audio dataset
!kaggle datasets download -d mmoreaux/environmental-sound-classification-50
!unzip environmental-sound-classification-50.zip




In [ ]:
import pandas as pd
csv_labels = pd.read_csv("./esc50.csv")
csv_labels.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


# Define labels

In [ ]:
# labels
labels=['cow', 'dog', 'sheep']


# path
base_path = './audio/audio/16000'

# output dir
output_base_path = './edge_dataset'

# Generate the dataset

In [ ]:
import shutil

if not os.path.exists(output_base_path):
    print ("Create dir " + output_base_path)
    os.mkdir(output_base_path)

filtered_labels = csv_labels[csv_labels.category.isin(labels)]
for x in range(len(filtered_labels)):
  label = filtered_labels.iloc[x]['category']
  filename = filtered_labels.iloc[x]['filename']
  print("Label: " + label)
  print("File name: "  + filename)
  output_path = output_base_path + "/" + label
  if not os.path.exists(output_path):
    print ("Create dir " + output_path)
    os.mkdir(output_path)
  src = base_path + "/" + filename
  dest = output_path + "/" + filename
  shutil.copyfile(src, dest)

   

# Download Keywords dataset and extract the noise dataset

In [ ]:
!wget https://cdn.edgeimpulse.com/datasets/keywords2.zip
# unzip the file
!unzip keywords2.zip -d ./other_dataset

In [ ]:
import librosa
import soundfile as sf
import numpy as np

# noise samples
noise_samples = 120

counter = 1

noise_input_path = "other_dataset/noise";

if not os.path.exists(output_base_path + "/noise/"):
    print ("Create dir " + output_base_path + "/noise/")
    os.mkdir(output_base_path + "/noise/")

for filename in os.listdir(noise_input_path):
  input_file_path = noise_input_path + "/" + filename
  output_file_path = output_base_path + "/noise/" + filename
  s, sr = librosa.load(input_file_path, sr=16000, mono=True)
 
  
  if (counter > noise_samples):
    break;

  counter = counter + 1;
  sf.write(output_file_path, s, 16000)



Create dir ./edge_dataset/noise/


# Edge Impluse Dataset Upload

## Install Edge Impulse CLI

In [ ]:
!npm install -g --unsafe-perm edge-impulse-cli

## Upload dataset

In [ ]:
# API Key
api_key = 'your_api_key'
# labels.append('noise')
labels = ['noise']
for label in labels:
  sample_dir = output_base_path + '/' + label + '/*.wav'
  file_list = []
  for filename in os.listdir(output_base_path + '/' + label ):
    _, ext = os.path.splitext(filename)
    if (ext.lower() == '.wav'):
     file_list.append(filename)
 
  print("Uploading files from ", sample_dir)
  print("N. Samples ", len(file_list))

  cmd = 'edge-impulse-uploader --api-key ' + api_key + ' --label ' + label + ' ' + sample_dir
  os.system(cmd)

print("Done!");

Uploading files from  ./edge_dataset/noise/*.wav
N. Samples  120
Done!


# Delete all dataset

In [ ]:
!rm -r ./edge_dataset